In [ ]:
# Import necessary libraries
# See tread in ChatGPT SAT/dSAT
# https://chatgpt.com/share/66e89de3-05c0-8007-8ee3-a728ff043b01
# This one for the HITS




import pandas as pd

# Load the spreadsheet to see the sheet names first
file_path = 'DATA dSAT.xlsx'
spreadsheet = pd.ExcelFile(file_path)

# Check the sheet names
spreadsheet.sheet_names

# Load the 'satmerged' sheet, skipping the first three rows for headers
df = pd.read_excel(file_path, sheet_name='satmerged', skiprows=3)

# Set the first two columns as part of the data
df.head()

# Re-read the data from the 'satmerged' sheet including the first 3 rows for headers so that we can process the data as described by the user.
df_raw = pd.read_excel(file_path, sheet_name='satmerged', header=[0, 1, 2])

# Display the first few rows to inspect the multi-level headers
df_raw.head()



# Filter the dataset to include only "HITS" columns
hits_data = df_raw.xs('HITS', level=0, axis=1)

# Add back the Phenotype and Animal No columns
hits_data.insert(0, 'Phenotype', df_raw['Phenotype'])
hits_data.insert(1, 'Animal No', df_raw['Animal No'])

# Reshape the data to long format
hits_long = hits_data.melt(id_vars=['Phenotype', 'Animal No'], 
                           var_name=['Stimulus Duration', 'Condition'], 
                           value_name='Hits')

# Display the reshaped data
import ace_tools as tools; tools.display_dataframe_to_user(name="Hits Long Format", dataframe=hits_long)


In [ ]:
# It seems the previous session has been reset. I will reload the data and organize it again before saving.

import pandas as pd

# Reload the dataset from the file
file_path = '/mnt/data/DATA dSAT.xlsx'

# Load the 'satmerged' sheet, skipping the first three rows for headers
df_raw = pd.read_excel(file_path, sheet_name='satmerged', header=[0, 1, 2])

# Filter the dataset to include only "HITS" columns
hits_data = df_raw.xs('HITS', level=0, axis=1)

# Add back the Phenotype and Animal No columns
hits_data.insert(0, 'Phenotype', df_raw['Phenotype'])
hits_data.insert(1, 'Animal No', df_raw['Animal No'])

# Reshape the data to long format
hits_long = hits_data.melt(id_vars=['Phenotype', 'Animal No'], 
                           var_name=['Stimulus Duration', 'Condition'], 
                           value_name='Hits')

# Save the reorganized data to an Excel file
output_file_path = "/mnt/data/updated_statmerged_data.xlsx"
hits_long.to_excel(output_file_path, index=False)

output_file_path


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import mixedlm

# Load the data (already cleaned)
df_raw = pd.read_excel('your_file_path.xlsx', sheet_name='satmerged', header=[0, 1, 2])

# Reshape the data to long format
hits_data = df_raw.xs('HITS', level=0, axis=1)
hits_data.insert(0, 'Phenotype', df_raw['Phenotype'])
hits_data.insert(1, 'Animal No', df_raw['Animal No'])

hits_long = hits_data.melt(id_vars=['Phenotype', 'Animal No'], 
                           var_name=['Stimulus Duration', 'Condition'], 
                           value_name='Hits')

# Convert columns to categorical type
hits_long['Phenotype'] = hits_long['Phenotype'].astype('category')
hits_long['Stimulus Duration'] = hits_long['Stimulus Duration'].astype('category')
hits_long['Condition'] = hits_long['Condition'].astype('category')

# Fit the mixed linear model
model = mixedlm("Hits ~ Phenotype * Stimulus_Duration * Condition", 
                data=hits_long, 
                groups=hits_long["Animal_No"])
mixed_lm_results = model.fit()
mixed_lm_results.summary()
    

In [ ]:

# Visualization of Hits data
plt.figure(figsize=(10, 6))
sns.pointplot(x='Stimulus_Duration', y='Hits', hue='Condition', 
              data=hits_long, dodge=True, markers=["o", "s"], 
              linestyles=["-", "--"], capsize=.1, ci="sd", palette="Set2", 
              hue_order=['SAT', 'dSAT'])

plt.title('Mean Hits by Stimulus Duration, Condition (All Phenotypes)')
plt.ylabel('Mean Hits')
plt.xlabel('Stimulus Duration (ms)')
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

# Faceted plot for each phenotype
g_hits = sns.catplot(x='Stimulus_Duration', y='Hits', hue='Condition', 
                     col='Phenotype', data=hits_long, kind='point', 
                     dodge=True, markers=["o", "s"], linestyles=["-", "--"], 
                     capsize=.1, ci="sd", palette="Set2", 
                     hue_order=['SAT', 'dSAT'])

g_hits.set_titles("Phenotype: {col_name}")
g_hits.set_axis_labels("Stimulus Duration (ms)", "Mean Hits")
g_hits.tight_layout()
plt.show()
    